In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [8]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         ret, frame = cap.read()

#         image, results = mediapipe_detection(frame, holistic)
#         draw_styled_landmarks(image, results)
#         cv2.imshow('OpenCV Feed', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [12]:
result_test = extract_keypoints(results)

In [13]:
result_test

array([ 0.51796788,  0.53401953, -1.4897182 , ...,  0.        ,
        0.        ,  0.        ])

In [14]:
DATA_PATH = os.path.join('MP_Data') 
actions = np.array(['hello', 'thanks', 'iloveyou'])
no_sequences = 30
sequence_length = 30
start_folder = 30

In [15]:
for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

In [16]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(start_folder, start_folder+no_sequences):
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [17]:
cap.release()
cv2.destroyAllWindows()

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
label_map = {label:num for num, label in enumerate(actions)}

In [20]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [21]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [22]:
np.array(sequences).shape

(180, 30, 1662)

In [23]:
np.array(labels).shape

(180,)

In [24]:
X = np.array(sequences)

In [25]:
X.shape

(180, 30, 1662)

In [26]:
y = to_categorical(labels).astype(int)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [28]:
y_test.shape

(9, 3)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [42]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
6/6 [==============================] - 7s 69ms/step - loss: 7.8969 - categorical_accuracy: 0.3158
Epoch 2/1000
6/6 [==============================] - 1s 97ms/step - loss: 15.0743 - categorical_accuracy: 0.3626
Epoch 3/1000
6/6 [==============================] - 0s 74ms/step - loss: 13.9518 - categorical_accuracy: 0.3567
Epoch 4/1000
6/6 [==============================] - 1s 87ms/step - loss: 23.5246 - categorical_accuracy: 0.3684
Epoch 5/1000
6/6 [==============================] - 0s 81ms/step - loss: 30.4453 - categorical_accuracy: 0.3567
Epoch 6/1000
6/6 [==============================] - 1s 88ms/step - loss: 91.8740 - categorical_accuracy: 0.3099
Epoch 7/1000
6/6 [==============================] - 1s 88ms/step - loss: 62.6761 - categorical_accuracy: 0.3743
Epoch 8/1000
6/6 [==============================] - 1s 84ms/step - loss: 45.9751 - categorical_accuracy: 0.3333
Epoch 9/1000
6/6 [==============================] - 1s 84ms/step - loss: 15.1116 - categorical_accuracy: 

6/6 [==============================] - 1s 83ms/step - loss: 8.9609 - categorical_accuracy: 0.3509
Epoch 74/1000
6/6 [==============================] - 1s 83ms/step - loss: 9.1413 - categorical_accuracy: 0.3567
Epoch 75/1000
6/6 [==============================] - 1s 85ms/step - loss: 9.4229 - categorical_accuracy: 0.3450
Epoch 76/1000
6/6 [==============================] - 1s 80ms/step - loss: 5.0704 - categorical_accuracy: 0.3743
Epoch 77/1000
6/6 [==============================] - 1s 85ms/step - loss: 4.3184 - categorical_accuracy: 0.3509
Epoch 78/1000
6/6 [==============================] - 1s 83ms/step - loss: 4.1172 - categorical_accuracy: 0.3333
Epoch 79/1000
6/6 [==============================] - 0s 73ms/step - loss: 3.5627 - categorical_accuracy: 0.2924
Epoch 80/1000
6/6 [==============================] - 0s 82ms/step - loss: 3.3709 - categorical_accuracy: 0.3450
Epoch 81/1000
6/6 [==============================] - 1s 86ms/step - loss: 2.6744 - categorical_accuracy: 0.2924
Epoch 

6/6 [==============================] - 0s 75ms/step - loss: 1.0525 - categorical_accuracy: 0.4561
Epoch 146/1000
6/6 [==============================] - 1s 80ms/step - loss: 1.0611 - categorical_accuracy: 0.4094
Epoch 147/1000
6/6 [==============================] - 1s 91ms/step - loss: 1.0501 - categorical_accuracy: 0.4854
Epoch 148/1000
6/6 [==============================] - 0s 74ms/step - loss: 1.0513 - categorical_accuracy: 0.3977
Epoch 149/1000
6/6 [==============================] - 1s 81ms/step - loss: 1.0454 - categorical_accuracy: 0.4386
Epoch 150/1000
6/6 [==============================] - 1s 80ms/step - loss: 1.0483 - categorical_accuracy: 0.4912
Epoch 151/1000
6/6 [==============================] - 1s 81ms/step - loss: 1.0587 - categorical_accuracy: 0.4678
Epoch 152/1000
6/6 [==============================] - 0s 73ms/step - loss: 1.0428 - categorical_accuracy: 0.4444
Epoch 153/1000
6/6 [==============================] - 1s 82ms/step - loss: 1.0622 - categorical_accuracy: 0.397

6/6 [==============================] - 0s 66ms/step - loss: 0.9293 - categorical_accuracy: 0.5146
Epoch 218/1000
6/6 [==============================] - 0s 62ms/step - loss: 0.9482 - categorical_accuracy: 0.5205
Epoch 219/1000
6/6 [==============================] - 0s 78ms/step - loss: 0.9253 - categorical_accuracy: 0.5497
Epoch 220/1000
6/6 [==============================] - 1s 85ms/step - loss: 0.8738 - categorical_accuracy: 0.6199
Epoch 221/1000
6/6 [==============================] - 0s 78ms/step - loss: 0.8935 - categorical_accuracy: 0.5497
Epoch 222/1000
6/6 [==============================] - 0s 77ms/step - loss: 0.9090 - categorical_accuracy: 0.5556
Epoch 223/1000
6/6 [==============================] - 1s 91ms/step - loss: 0.8969 - categorical_accuracy: 0.5673
Epoch 224/1000
6/6 [==============================] - 0s 77ms/step - loss: 0.8688 - categorical_accuracy: 0.5614
Epoch 225/1000
6/6 [==============================] - 1s 87ms/step - loss: 0.8775 - categorical_accuracy: 0.584

6/6 [==============================] - 1s 81ms/step - loss: 0.8379 - categorical_accuracy: 0.6374
Epoch 290/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8169 - categorical_accuracy: 0.5965
Epoch 291/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.8621 - categorical_accuracy: 0.6257
Epoch 292/1000
6/6 [==============================] - 0s 73ms/step - loss: 0.8582 - categorical_accuracy: 0.5965
Epoch 293/1000
6/6 [==============================] - 0s 73ms/step - loss: 0.8902 - categorical_accuracy: 0.5906
Epoch 294/1000
6/6 [==============================] - 0s 83ms/step - loss: 0.8302 - categorical_accuracy: 0.6257
Epoch 295/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8254 - categorical_accuracy: 0.6082
Epoch 296/1000
6/6 [==============================] - 1s 82ms/step - loss: 0.8818 - categorical_accuracy: 0.5439
Epoch 297/1000
6/6 [==============================] - 0s 84ms/step - loss: 0.8754 - categorical_accuracy: 0.567

...

6/6 [==============================] - 0s 81ms/step - loss: 0.8617 - categorical_accuracy: 0.5848
Epoch 650/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.8058 - categorical_accuracy: 0.6550
Epoch 651/1000
6/6 [==============================] - 0s 75ms/step - loss: 0.8858 - categorical_accuracy: 0.5848
Epoch 652/1000
6/6 [==============================] - 0s 72ms/step - loss: 0.8317 - categorical_accuracy: 0.6199
Epoch 653/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.8001 - categorical_accuracy: 0.6023
Epoch 654/1000
6/6 [==============================] - 0s 75ms/step - loss: 0.8407 - categorical_accuracy: 0.6433
Epoch 655/1000
6/6 [==============================] - 0s 69ms/step - loss: 0.8333 - categorical_accuracy: 0.6023
Epoch 656/1000
6/6 [==============================] - 1s 91ms/step - loss: 0.7742 - categorical_accuracy: 0.6433
Epoch 657/1000
6/6 [==============================] - 0s 77ms/step - loss: 0.8094 - categorical_accuracy: 0.602

6/6 [==============================] - 1s 84ms/step - loss: 0.8560 - categorical_accuracy: 0.5731
Epoch 722/1000
6/6 [==============================] - 1s 90ms/step - loss: 0.9264 - categorical_accuracy: 0.5497
Epoch 723/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8334 - categorical_accuracy: 0.6433
Epoch 724/1000
6/6 [==============================] - 1s 81ms/step - loss: 0.7998 - categorical_accuracy: 0.5906
Epoch 725/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8012 - categorical_accuracy: 0.6491
Epoch 726/1000
6/6 [==============================] - 1s 81ms/step - loss: 0.8355 - categorical_accuracy: 0.6257
Epoch 727/1000
6/6 [==============================] - 0s 80ms/step - loss: 0.8219 - categorical_accuracy: 0.6374
Epoch 728/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8084 - categorical_accuracy: 0.6550
Epoch 729/1000
6/6 [==============================] - 1s 84ms/step - loss: 0.7899 - categorical_accuracy: 0.637

6/6 [==============================] - 1s 79ms/step - loss: 0.8841 - categorical_accuracy: 0.5789
Epoch 794/1000
6/6 [==============================] - 0s 75ms/step - loss: 0.8323 - categorical_accuracy: 0.6199
Epoch 795/1000
6/6 [==============================] - 0s 79ms/step - loss: 0.8620 - categorical_accuracy: 0.6316
Epoch 796/1000
6/6 [==============================] - 0s 79ms/step - loss: 0.9092 - categorical_accuracy: 0.5906
Epoch 797/1000
6/6 [==============================] - 1s 90ms/step - loss: 0.9313 - categorical_accuracy: 0.5614
Epoch 798/1000
6/6 [==============================] - 1s 80ms/step - loss: 0.8813 - categorical_accuracy: 0.6433
Epoch 799/1000
6/6 [==============================] - 0s 78ms/step - loss: 0.9765 - categorical_accuracy: 0.4912
Epoch 800/1000
6/6 [==============================] - 0s 75ms/step - loss: 0.8842 - categorical_accuracy: 0.5556
Epoch 801/1000
6/6 [==============================] - 1s 85ms/step - loss: 0.9094 - categorical_accuracy: 0.578

6/6 [==============================] - 0s 71ms/step - loss: 0.9185 - categorical_accuracy: 0.5906
Epoch 866/1000
6/6 [==============================] - 0s 76ms/step - loss: 0.9463 - categorical_accuracy: 0.5848
Epoch 867/1000
6/6 [==============================] - 0s 72ms/step - loss: 0.7847 - categorical_accuracy: 0.6842
Epoch 868/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.7948 - categorical_accuracy: 0.6316
Epoch 869/1000
6/6 [==============================] - 0s 73ms/step - loss: 0.7747 - categorical_accuracy: 0.6608
Epoch 870/1000
6/6 [==============================] - 1s 83ms/step - loss: 0.8455 - categorical_accuracy: 0.6023
Epoch 871/1000
6/6 [==============================] - 0s 76ms/step - loss: 0.8105 - categorical_accuracy: 0.6374
Epoch 872/1000
6/6 [==============================] - 1s 85ms/step - loss: 0.8148 - categorical_accuracy: 0.6608
Epoch 873/1000
6/6 [==============================] - 1s 85ms/step - loss: 0.8708 - categorical_accuracy: 0.567

6/6 [==============================] - 0s 80ms/step - loss: 0.8148 - categorical_accuracy: 0.6608
Epoch 938/1000
6/6 [==============================] - 1s 88ms/step - loss: 0.7867 - categorical_accuracy: 0.6784
Epoch 939/1000
6/6 [==============================] - 1s 84ms/step - loss: 0.8061 - categorical_accuracy: 0.6374
Epoch 940/1000
6/6 [==============================] - 1s 88ms/step - loss: 0.8918 - categorical_accuracy: 0.5789
Epoch 941/1000
6/6 [==============================] - 0s 79ms/step - loss: 0.8361 - categorical_accuracy: 0.6316
Epoch 942/1000
6/6 [==============================] - 1s 85ms/step - loss: 0.9694 - categorical_accuracy: 0.5439
Epoch 943/1000
6/6 [==============================] - 1s 82ms/step - loss: 0.8761 - categorical_accuracy: 0.5556
Epoch 944/1000
6/6 [==============================] - 0s 74ms/step - loss: 0.8369 - categorical_accuracy: 0.6316
Epoch 945/1000
6/6 [==============================] - 0s 72ms/step - loss: 0.7499 - categorical_accuracy: 0.684

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [46]:
res = model.predict(X_test)

1/1 [==============================] - 1s 591ms/step


In [47]:
actions[np.argmax(res[4])]

'thanks'

In [48]:
actions[np.argmax(y_test[4])]

'iloveyou'

In [49]:
model.save('action.h5')

In [50]:
model.load_weights('action.h5')

In [51]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [52]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 44ms/step


In [53]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [54]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 2],
        [1, 0]],

       [[3, 4],
        [1, 1]],

       [[3, 0],
        [4, 2]]], dtype=int64)

In [55]:
accuracy_score(ytrue, yhat)

0.3333333333333333

In [56]:
from scipy import stats

In [57]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [70]:
#prediction
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]


            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
  
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 28ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
hello
<class 'mediapipe.pyth

1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.pyth

1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.pyth

1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapip

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===

1/1 [==============================] - 0s 24ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapi

1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
hello
<class 'mediapipe.pyth

In [68]:
import tensorflow as tf

model = tf.keras.models.load_model('action.h5')


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False 
tflite_model = converter.convert()


with open('action.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\AADYAD~1\AppData\Local\Temp\tmpemlfyvnf\assets


INFO:tensorflow:Assets written to: C:\Users\AADYAD~1\AppData\Local\Temp\tmpemlfyvnf\assets


In [63]:
import _pickle as pickle

In [67]:
file_path = "label_map.txt"

with open(file_path, "w") as file:
    for key, value in label_map.items():
        file.write(f"{key}: {value}\n")
